In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import glob
import shutil
import csv

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# for callback
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

### Preparing the data
#### Preparing training and validation set

In [2]:
def split_data(path_to_data, path_to_save_train, path_to_save_val, split_size=0.1) :
    folders = os.listdir(path_to_data) # la liste des dossiers disponible au chemin donné
    for folder in folders :
        full_path = os.path.join(path_to_data, folder) # pour avoir le chemin complet en ajoutant le nom des dossiers
        images_paths = glob.glob(os.path.join(full_path, '*.png')) # ca prend tous les fichiers à l'intérieur du dossier et les télécharge (le join il va a chaque fois ajouter le path du dossier et ajoutant le nom du fichier) ca nous retourne une liste d'images
        x_train, x_val = train_test_split(images_paths, test_size=split_size) # split en train et validation

        for x in x_train : 
            path_to_folder = os.path.join(path_to_save_train, folder) # pour recréer les même dossier que dans le dossier de base
            if not os.path.isdir(path_to_folder) : 
                os.makedirs(path_to_folder) # si il n'existe pas il le crée
            shutil.copy(x, path_to_folder)
        
        for x in x_val : 
            path_to_folder = os.path.join(path_to_save_val, folder) # pour recréer les même dossier que dans le dossier de base
            if not os.path.isdir(path_to_folder) : 
                os.makedirs(path_to_folder) # si il n'existe pas il le crée
            shutil.copy(x, path_to_folder)


In [3]:
if __name__=='__main__' :
    path_to_data = './Train'
    path_to_save_train = './Training_data/train'
    path_to_save_val = './Training_data/val'
    split_data(path_to_data, path_to_save_train, path_to_save_val)

#### Preparing the test set
Pour avoir les labels de chaque images de test puisqu'elles sont toute dans un même dossier, toute mélangée

In [4]:
def order_test_set(path_to_images, path_to_csv) :
    try : 
        with open(path_to_csv, 'r') as csvfile : 
            reader = csv.reader(csvfile, delimiter=',')
            for i, row in enumerate(reader) :
                if i == 0 : 
                    continue # pour ne pas prendre en compte la première ligne
                img_name = row[-1].replace('Test/','') # prendre la dernière colonne qui est le nom de l'image en enlevant le 'Test/' au début de chaque nom d'image
                label = row[-2]

                path_to_folder = os.path.join(path_to_images,label) # on crée un dossier avec le nom du label comme le dataset d'entrainement
                if not os.path.isdir(path_to_folder) :
                    os.makedirs(path_to_folder)

                img_full_path = os.path.join(path_to_images, img_name)

                shutil.move(img_full_path, path_to_folder) # on va déplacer et pas copier
    except : 
        print("On ne peut pas ouvrir le fichier csv") 

In [5]:
path_to_images = './Test'
path_to_csv = './Test.csv'
order_test_set(path_to_images, path_to_csv)

On ne peut pas ouvrir le fichier csv


### Réseau de neurones
Maintenant que le dataset est prêt on peut maintenant passer à la conception du modèle de classification

In [2]:
def streetsigns_model(nbr_classes) :
    my_input = Input(shape=(60,60, 3)) # taille moyenne et les 3 canaux RGB
    x= Conv2D(32, (3,3), activation='relu')(my_input)
    x= Conv2D(64, (3,3), activation='relu')(x)
    x= MaxPool2D()(x)
    x= BatchNormalization()(x)

    x= Conv2D(128, (3,3), activation='relu')(x)
    x= MaxPool2D()(x)
    x= BatchNormalization()(x)

    x= GlobalAvgPool2D()(x)
    #x = Flatten()(x)
    x= Dense(64, activation='relu')(x)
    x= Dense(43, activation='softmax')(x)
    model = Model(inputs=my_input, outputs=x)
    return model

In [3]:
if __name__=='__main__' :

    model = streetsigns_model(43)
    model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 58, 58, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 28, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 128)       73856 

On peut ainsi voir ainsi l'architecture de notre réseaux de neurones et surtout voir le nombre de paramètres à entrainer et tout. 

Remarque : on peut voir qu'on a bien moins de parmaètres avec global Average que Flatten 

### Générateur de données 
Pour alimenter notre modèle pour l'entrainement et ensuite le test, il contiendra : 
- préprocessing de toutes les images
- préparation des images pour l'entrainement 
- acheminement des images

In [5]:
def create_generators(batch_size, train_data_path, val_data_path, test_data_path) :
    preprocessor = ImageDataGenerator(
        rescale = 1/255. # pour assurer une division flottante
    )

    # !très important pour prendre des données en prenant chaque sous dossier comme classe à part entière
    train_generator = preprocessor.flow_from_directory(
        train_data_path,
        class_mode="categorical",
        target_size=(60,60), #resize all images
        color_mode = 'rgb', # type d'images
        shuffle = True,
        batch_size=batch_size
    )

    val_generator = preprocessor.flow_from_directory(
        val_data_path,
        class_mode="categorical",
        target_size=(60,60), #resize all images
        color_mode = 'rgb', # type d'images
        shuffle = False,
        batch_size=batch_size
    )

    test_generator = preprocessor.flow_from_directory(
        test_data_path,
        class_mode="categorical",
        target_size=(60,60), #resize all images
        color_mode = 'rgb', # type d'images
        shuffle = False,
        batch_size=batch_size
    )

    return train_generator, val_generator, test_generator

In [6]:
train_data_path = './Training_data/train'
val_data_path = './Training_data/val'
test_data_path = './Test'
batch_size = 64

train_generator, val_generator, test_generator= create_generators(batch_size, train_data_path, val_data_path, test_data_path)

nbr_classes = train_generator.num_classes

Found 39209 images belonging to 43 classes.
Found 10632 images belonging to 43 classes.
Found 12630 images belonging to 43 classes.


In [7]:
model = streetsigns_model(nbr_classes)

### Fitting du modèle et sauvegarde du meilleur 

On va préparer le modèle comme précédemment et pour la sauvegarde on utilise quelque chose qui s'appelle un Callback qui va permettre de sauvegarder le meilleur modèle.

Pour un grand nombre d'époques il y a le Earlystopping pour voir si le modèle ne s'améliore pas on s'arrête

In [8]:
# callbacks
path_to_save_model = './Models'
ckpt_saver = ModelCheckpoint(
    path_to_save_model,
    monitor='val_accuracy', # sur quoi on se base pour voir le meilleur
    mode = 'max', # max de l'accuracy sur la validation
    save_best_only = True,
    save_freq='epoch', # ne voit qu'à la fin de l'époque
    verbose=1
) 

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10 # après 10 époques ca change pas on s'arrête
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # on choisit categorical_crossentropy car dans les générateurs on a défini categorical comme class_mode

epochs = 15

In [19]:
# Le générateur contient autant les images que les labels
model.fit(
    train_generator,
    epochs = epochs,
    batch_size = batch_size,
    validation_data = val_generator,
    callbacks=[ckpt_saver, early_stop]
)

Epoch 1/15
613/613 [==============================] - ETA: 0s - loss: 2.4095 - accuracy: 0.3206
Epoch 1: val_accuracy improved from -inf to 0.13732, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
613/613 [==============================] - 537s 874ms/step - loss: 2.4095 - accuracy: 0.3206 - val_loss: 3.0621 - val_accuracy: 0.1373
Epoch 2/15
613/613 [==============================] - ETA: 0s - loss: 1.1022 - accuracy: 0.6767
Epoch 2: val_accuracy improved from 0.13732 to 0.74859, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
613/613 [==============================] - 569s 928ms/step - loss: 1.1022 - accuracy: 0.6767 - val_loss: 0.7952 - val_accuracy: 0.7486
Epoch 3/15
613/613 [==============================] - ETA: 0s - loss: 0.4014 - accuracy: 0.9001
Epoch 3: val_accuracy improved from 0.74859 to 0.91930, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
613/613 [==============================] - 475s 7

L'entrainement fait et terminé on a enregistré notre modèle, on pourra alors l'utiliser pour son évaluation en chargeant les données enregistrées dans le dossier Models

In [11]:
model = tf.keras.models.load_model('./Models')
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 58, 58, 32)        896       
                                                                 
 conv2d_10 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_6 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_11 (Conv2D)          (None, 26, 26, 128)       7385

In [11]:
model.evaluate(test_generator, batch_size=64)

198/198 [==============================] - 57s 284ms/step - loss: 0.3312 - accuracy: 0.9170


[0.3312202990055084, 0.9170229434967041]

#### Améliorations du modèle
Pour améliorer le modèle on a plusieurs possibilités : 
- Changer la taille du Batch
- Augmenter (ou diminuer) le nombre d'époques
- Changer l'architecture du modèle (changer les couches ou en ajouter ou diminuer)
- Dans la création du générateur (dans la partie ImageDataGenerator) il y a un certain nombre de techniques pour de la data augmentation (surtout dans le domaine du traitement d'images) 
- On peut mettre en place plusieurs pré-processeurs pour les adapter, chacun à une partie du problème (train et pas validation et test par exemple) surtout dans le cas de l'augmentation des données (avec des shifts et des zoom) 
- On peut changer l'optimize en utilisant opitmizer = tf.keras.optimizers.NomOptimizer() et on choisit d'après la documentation qu'on a
- Ajouter et changer le learning rate et l'ajouter à l'optimizer
#### Essayer notre modèle sur une photo à part entière
Pour le déploiement du modèle et pour se faire on peut créer un nouveau fichier qui fera juste cela

In [12]:
def predict_with_model(model, img_path) : 
    image = tf.io.read_file(img_path) # On lit l'image
    image = tf.image.decode_png(image, channels=3) # On la décode
    image = tf.image.convert_image_dtype(image, dtype=tf.float32) # Convertir les entiers en float, cela permet de rescaler nos images : A VOIR !
    image = tf.image.resize(image , [60,60]) # resize les images et on a alors de la forme (60,60,3)
    image = tf.expand_dims(image, axis=0) # pour avoir au final (1,60,60,3) pour l'adapter à notre modèle (voir summary du modèle il attend un tel format)

    prediction = model.predict(image) # peut etre une décision ou un ensemble de probabilités (pour chaque classe)
    prediction = np.argmax(prediction) # Pour avoir l'indexe de la meilleure probabilité et par conséquent le label
    return prediction

if __name__=='__main__' :

    img_path = "./Test/0/00579.png"
    model = tf.keras.models.load_model('./Models')
    prediction = predict_with_model(model, img_path)
    print(f'prediction : {prediction}')

prediction : 0
